## HSTaXe Cookbook: Extraction for ACS/WFC Subarray G800L Data

This notebook contains a step-by-step guide for performing a basic spectral extraction with HSTaXe for G800L subaaray data from ACS/WFC.

## 1. HSTaXe Installation

Follow instructions from https://github.com/spacetelescope/hstaxe to install HSTaXe software

## 2. Cookbook Data

 The example data we use in this notebook are available [here](https://stsci.app.box.com/folder/196022333710). Keep this notebook and the 'cookbook_data' folder in the same directory.

## 3. Embedding Subarray in a Full Frame

The HSTaXe software will not run on a subarray data so we need to embed subarray in a full frame. The example data in the 'cookbook_data' folder is embedded so it can be directly used to run this notebook. But for the future reference, there is a code `embed_subarray_acs.py` ([here](https://stsci.app.box.com/folder/196022333710)) which can be used for this purpose. The code requires original subarray FLT files and corresponding SPT files. 

## 4. Load a Few Python Modules

In [ ]:
%matplotlib inline
import glob
from astropy.io import fits
import numpy as np
import os, shutil
import matplotlib.pyplot as plt

from drizzlepac import astrodrizzle
from hstaxe import axetasks
from stwcs import updatewcs

In [ ]:
# Check hstaxe version
import hstaxe
hstaxe.__version__

We save the current working directory

In [ ]:
cwd = os.getcwd()
print("The current directory is %s" % (cwd))

## 5. Copying Data from the 'cookbook_data' Folder

We will create a G800L subdirectory to copy all of the G800L files into. This where we will prepare the G800L data.

Creating the directory, removing any existing one

In [ ]:
os.chdir(cwd)
if os.path.isdir("G800L"):
    shutil.rmtree("G800L")
os.mkdir("G800L")

Copying the G800L data (which we grab from our cookbook_data directory)

In [ ]:
os.system("cp cookbook_data/G800L/*flt.fits G800L/")
os.system("cp cookbook_data/G800L/G800L.lis G800L/")

We move into the G800L directory and check the content of the G800L.lis file **(in this case, we have only 1 image but you could have multiple images)**

In [ ]:
os.chdir(cwd)
os.chdir("G800L")
!cat G800L.lis

print()
os.system(f"fitsheader -f -e 0 -k APERTURE -k FILTER1 *fits")

We will create a F775W subdirectory to copy all of the F775W files into. This where we will prepare the F775W data.

Creating the directory, removing any existing one

In [ ]:
os.chdir(cwd)

if os.path.isdir("F775W"):
    shutil.rmtree("F775W")

os.mkdir("F775W")

Copy the F775W data (which we grab from our cookbook_data directory)

In [ ]:
os.system("cp cookbook_data/F775W/*flt.fits F775W/")
os.system("cp cookbook_data/F775W/F775W.lis F775W/")

We move into the F775W directory and check the content of the F775W.lis file 

In [ ]:
os.chdir(cwd)
os.chdir("F775W")
!cat F775W.lis

print()
os.system(f"fitsheader -f -e 0 -k APERTURE -k FILTER1 *fits")

## 6. Check/Verify Matching WCS Information

It is possible that the WCS in the direct and grism images differ. In this section we will use a function to process all the direct and grism images to verify that the WCS information is consistent throughout. If there is any disagreement in WCS information we call `updatewcs` with the database keyword set to False, which will roll back all the solutions to the original distortion-corrected WCS. For more information regarding HST WCS and improved absolute astrometry please see [WFC3 Instrument Science Report 2022-06 (Mack et al. 2022)](https://ui.adsabs.harvard.edu/abs/2022wfc..rept....6M/abstract). For documentations on `updatewcs` please see [here](https://stwcs.readthedocs.io/en/latest/updatewcs.html).

Before running _updatewcs_, we need to [set CRDS environment variables](https://hst-crds.stsci.edu/docs/cmdline_bestrefs/). We will point to a subdirectory called 'crds_cache/' using the JREF environment variable. The JREF variable is used for ACS reference files and different instruments use other variables, e.g., IREF for WFC3. You have the option to permanently add these environment variables to your user profile by adding the path in your shell's configuration file. If you're using bash, you would edit the '~/.bash_profile' file with lines such as:

- export CRDS_PATH="$HOME/crds_cache"

- export CRDS_SERVER_URL="https://hst-crds.stsci.edu"

- export iref="${CRDS_PATH}/references/hst/iref/"

If you have already set up the CRDS environment variables you may skip running the cell below.

In [ ]:
os.environ["CRDS_SERVER_URL"] = "https://hst-crds.stsci.edu"
os.environ['CRDS_SERVER'] = 'https://hst-crds.stsci.edu'
if "CRDS_PATH" not in os.environ.keys():
    os.environ["CRDS_PATH"] = os.path.join(os.environ["HOME"],"crds_cache")
if "jref" not in os.environ.keys():
    os.environ["jref"] = "$HOME/crds_cache/references/hst/jref/"

In [ ]:
def check_wcs(images):
    """ A helper function to verify the active world coordinate solutions match and roll them back if they do not 
    (written by the WFC3 team -- Aidan Pidgeon, Benjamin Kuhn, Debopam Som)
    
    Parameter
    ---------
    images : list 
        a list of grism and direct images 
        
    Return
    ------
    N/A
    """
    
    direct_wcs = []
    grism_wcs = []

    for f in images:
        # get filter for image to distinguish between direct and grism
        filt = fits.getval(f, 'FILTER1')
       
        hdul = fits.open(f)
        db_bool = 'WCSCORR' not in hdul
        hdul.close()
        
        try:
            # get the active solution from the file's "SCI" extension
            wcsname = fits.getval(f, 'WCSNAME', ext=('SCI', 1))
            if db_bool == True:
                updatewcs.updatewcs(f,use_db=db_bool)
        except KeyError:
            updatewcs.updatewcs(f,use_db=db_bool)
            wcsname = fits.getval(f, 'WCSNAME', ext=('SCI', 1))
        
        # seperate between direct and grism
        if 'G' in filt:
            grism_wcs.append(wcsname)
        if 'F' in filt:
            direct_wcs.append(wcsname)

    # get the number of unique active solutions in the direct and grism images       
    num_wcs_direct = len(set(direct_wcs))
    num_wcs_grism = len(set(grism_wcs))
    
    # roll back WCS on all files if there is more than one active solution for either direct or grism images
    if num_wcs_direct > 1 or num_wcs_grism > 1:
        [updatewcs.updatewcs(file,use_db=False) for file in images]
        print('WCS reset complete')

    # roll back WCS on all files if the active solution for the direct images do not match the grism images
    elif set(direct_wcs) != set(grism_wcs):
        [updatewcs.updatewcs(file,use_db=False) for file in images]
        print('WCS reset complete')

    # do nothing if there is one unique active solution and they match
    elif set(direct_wcs) == set(grism_wcs):
        print(f"No WCS update needed. All grism and direct images use WCS: {grism_wcs[0]}.")

In [ ]:
os.chdir(cwd)
all_images = glob.glob('F775W/*_flt.fits')+\
            glob.glob('G800L/*_flt.fits')

# to populate 'crds_cache' folder with all reference files
for image in all_images:
    command_line_input = f'crds bestrefs --files {image} --sync-references=1 --update-bestrefs'
    os.system(command_line_input)

# or to get only IDCTAB and MDRIZTAB files
#for image in all_images:
#    command_line_input = f'crds bestrefs --files {image} --types IDCTAB MDRIZTAB --sync-references=1 --update-bestrefs'
#    os.system(command_line_input)    

check_wcs(all_images)

## 7. Drizzling the Input Data

We now create a G800L mosaic using the G800L data  **(in this case, only 1 image is drizzled but you could have multiple images)**

Note, in the cell below we have set the AstroDrizzle parameters for processing only a single image. **If you have more than one direct image to drizzle together please set the parameters appropriately.** For example `driz_separate`, `driz_sep_wcs`, `median, blot`, and `driz_cr` should all be set to True. For more information please see the AstroDrizzle documentation [here](https://drizzlepac.readthedocs.io/en/latest/astrodrizzle.html). Finally, if your input images were FLC images rather than FLT images, change the extension to 'drc.fits' in the cell below.

This mosaic will be used to set up the proper astrometry for each individual FLT files. We can only extract G800L spectra from FLT files which have been used to make this mosaic

In [ ]:
os.chdir(cwd)
os.chdir("G800L")
astrodrizzle.AstroDrizzle("@G800L.lis", output="G800L", build=True, mdriztab=True, in_memory=False, 
                          preserve=False, skysub=False, driz_separate=False, median=False, 
                          blot=False, driz_cr=False, driz_sep_wcs=False,
                          driz_combine=True, final_wcs=False)

We already created a mosaic of all the G800L data for astrometric purposes, and we now create an F775W mosaic using the G800L mosaic as the astrometric reference frame. This will ensure that the G800L and F775W mosaics have pixels with the same RA and DEC. 

We create a F775W mosaic using the F775W data and the G800L mosaic as a reference **(in this case, only 1 image is drizzled)**

Note, in the cell below we have set the AstroDrizzle parameters for processing only a single image. **If you have more than one direct image to drizzle together please set the parameters appropriately.** For example `driz_separate`, `driz_sep_wcs`, `median, blot`, and `driz_cr` should all be set to True. For more information please see the AstroDrizzle documentation [here](https://drizzlepac.readthedocs.io/en/latest/astrodrizzle.html). Finally, if your input images were FLC images rather than FLT images, change the extension to 'drc.fits' in the cell below.

In [ ]:
os.chdir(cwd)
os.chdir("F775W")
ref = "../G800L/G800L_drz.fits[1]"
astrodrizzle.AstroDrizzle("@F775W.lis", output="F775W", build=True, mdriztab=True, in_memory=False, 
                          preserve=False, skysub=True, driz_separate=False, median=False, 
                          blot=False, driz_cr=False, driz_sep_wcs=False, 
                          driz_combine=True, final_wcs=True, driz_sep_refimage=ref, final_refimage=ref)

The F775W and G800L should be aligned and bright objects should generate bright spectra in the expected position. We should see very liittle offset in the y-direction for ACS grism data

In [ ]:
os.chdir(cwd)
plt.rcParams["figure.figsize"] = (10,7)
plt.subplot(1,2,1)
d = fits.open("F775W/F775W_drz.fits")[1].data
im1 = plt.imshow(d,origin="lower")
im1.set_clim(0,8.5)

plt.subplot(1,2,2)
d = fits.open("G800L/G800L_drz.fits")[1].data
im1 = plt.imshow(d,origin="lower")
im1.set_clim(0,0.85)

## 8. Creating a Catalog with SExtractor

We create an object catalog using sextractor

This is one step that needs to be done carefully as several things can go wrong.

- Make sure you set the magnitude zeropoint properly for the image you are using
- One can generate a simple catalog using:

sex -c default.sex F775W_drc.fits[1] -DETECT_THRESH 5 -MAG_ZEROPOINT 25.656

or use a full command as given in 'run_sext.e' file (cookbook_data/catalog)

- See default.param for the required parameters that aXe will be looking for.
- Check the resulting regions file and catalog to make sure that all objects have good magnitudes (i.e. no mag of 99.)
- Edit F775W.cat and rename column 'MAG_AUTO' with 'MAG_F7692', or you will get an "aXeError: Catalogue: test.cat does not contain any magnitude column!" error when running iolprep

This catalog, when doing a simple extraction, will be used to compute the SED of each sources. These SEDs will be used to compute our contamination models. In this example, we used a single band, F775W, but we could have added information in other bands such as F814W or F606W for example. This requires running Sextractor in matched photometry mode, and the creation of a catalog where magnitudes in multiple bands are properly listed

For simplicity, here, we copy an already generated catalog:

In [ ]:
os.chdir(cwd)
os.system("cp cookbook_data/catalog/F775W.cat ./F775W/")
!cat ./F775W/F775W.cat

## 9. Running HSTaXe

We can now run HSTaXe. We start by setting up some necessary environment variables that point to the various HSTaXe directories. **Make sure that you keep the path length to be less than 80 characters when the code points to individual data files, or about 60 characters without the file names to be safe**

Create a directory called CONF and copy the ACS G800L Calibration files in there.

In [ ]:
os.chdir(cwd)
if os.path.isdir("CONF"):
    shutil.rmtree("CONF")
os.mkdir("CONF")

os.system("cp cookbook_data/CONF/* CONF/")

Set up some work directories and environment variables required by HSTaXe:

In [ ]:
os.chdir(cwd)
import os

if os.path.isdir("DATA"):
    shutil.rmtree("DATA")
os.mkdir("DATA")
os.environ['AXE_IMAGE_PATH'] = cwd+'/DATA/' 
print ('--> variable AXE_IMAGE_PATH   set to '+cwd+'/DATA/')

os.environ['AXE_CONFIG_PATH'] = cwd+'/CONF/'
print ('--> variable AXE_CONFIG_PATH  set to '+cwd+'/CONF/')

if os.path.isdir("OUTPUT"):
    shutil.rmtree("OUTPUT")
os.mkdir("OUTPUT")
os.environ['AXE_OUTPUT_PATH'] = cwd+'/OUTPUT/'
print ('--> variable AXE_OUTPUT_PATH  set to '+cwd+'/OUTPUT/')

if os.path.isdir("DRIZZLE"):
    shutil.rmtree("DRIZZLE")
os.mkdir("DRIZZLE")
os.environ['AXE_DRIZZLE_PATH'] = cwd+'/DRIZZLE/' 
print ('--> variable AXE_DRIZZLE_PATH  set to '+cwd+'/DRIZZLE/')

os.mkdir("DRIZZLE/tmp")

print ("Length of AXE_IMAGE_PATH is",len(os.environ['AXE_IMAGE_PATH']),"characters")

We define the FOV boundaries for the ACS observations.

- Four numbers to specify the modifications

   [left, right, bottom, top] to the target area on the input
   images. E.g. 100,500,10,0 would include in the Input Object
   Lists all objects with -100 < x < x_size + 500 and
   -10 < y < y_size.
   
- at present using 0,0,0,0 -- **it is only important when objects are at the edges** 


In [ ]:
dimension_info = "0,0,0,0"

We copy the G800L FLT files and the F775W FLT files in the DATA directory

You can either use the original data or optionally the FLT files used to create the G800L mosaic earlier, which will have some extra bad pixels flagging

In [ ]:
os.chdir(cwd)
os.system("cp G800L/*flt.fits DATA/")
os.system("cp F775W/*flt.fits DATA/")

We use the iolprep aXe task to generate individual F775W catalogs

This task will create the individual F775W extraction catalogs, TWO (1 for chip1 and 1 for chip2) for each of the files listed in the F775W.lis file. We pass the F775W mosaic to it, as it contains all the information about all the individual F775W FLT file. 

Here, the catalog name is 'filename_flt_1.cat' -- **here we are using only 1 chip because of the subarray data**

In [ ]:
os.chdir(cwd)
os.chdir("F775W")

axetasks.iolprep(drizzle_image='F775W_drz.fits',
                 input_cat='F775W.cat',
                 dimension_in=dimension_info)

We copy the newly create catalog files into the DATA directory

In [ ]:
os.chdir(cwd)
os.system("cp ./F775W/j*.cat DATA/")

In [ ]:
!ls ./DATA

We are almost ready to extract the spectra. We need to create an file aXe.lis containing the G800L images, expected catalog names and associated F775W direct images

The G800L mosaic we created earlier is not used directly during the aXe extraction process. However, the F775W mosaic was used to create an object master catalog. This catalog will be processed to generate individual object catalogs for the files used to create the F775W mosaic. The aXe.lis file lists which F775W images are logically associated with a particular G800L image. Ideally, these are images taken in the same HST visit so that we can be sure that the WCS of both files are consistent.

The aXe.lis file is a simple text file, with a slightly different format than the one above. In this file, each line contains 3 items:

- The name of a G800L FLT file (e.g. [grism_rootname]_flt.fits)
- One catalog name corresponding to one chip on ACS/WFC 
- The name of the direct imaging file [direct_rootname]_flt.fits associated with the G800L data and the catalog.



In [ ]:
os.chdir(cwd)
os.system("cp cookbook_data/aXe.lis .")
!cat aXe.lis

We run aXeprep. This task will amongst other things take care of background subtracting the G800L data using a single master sky. First we check which Chip we are using and what configuration files are needed.

In [ ]:
os.chdir(cwd)

fd=fits.open('./DATA/jdql01jiq_flt.fits')
ccdchip = fd[1].header['CCDCHIP']
ccdamp = fd[0].header['CCDAMP']
print('CCD_CHIP =',ccdchip)
print('CCD_AMP =',ccdamp)

if ccdchip == 1:
    config_file='ACS.WFC.CHIP1.Cycle13.5.conf'
    msky_file='ACS.WFC.CHIP1.msky.1.smooth.fits'
else:
    config_file='ACS.WFC.CHIP2.Cycle13.5.conf'
    msky_file='ACS.WFC.CHIP2.msky.1.smooth.fits'

print('Config_File =',config_file)
print('MSky_File =',msky_file)

In [ ]:
os.chdir(cwd)

axetasks.axeprep(inlist="aXe.lis",
                     configs=config_file,
                     backgr=True,
                     backims=msky_file,
                     norm=True,
                     mfwhm=3.0)

**We can now proceed with a simple box extraction of our G800L spectra.** This will not combine individual 1D spectra and we create one extracted spectrum per object and get G800L FLT file we are processing. The contamination is estimated using the Gaussian model of each object that is included in the SExtractor object catalog.

For each of the G800L input FLT file, this will create the following in the OUTPUT/ directory:

- [rootname]_flt_2.cat : Object catalog for the FLT file [rootname]_flt.fits
- [rootname]_flt_2.OAF : Aperture file
- [rootname]_flt_2.PET.fits : The Pixel Extraction Table, containing all the unbinned information about each spectrum
- [rootname]_flt_2.SPC.fits : 1D extracted spectra
- [rootname]_flt_2.CONT.fits : Contamination estimate for eact of the spectra
- [rootname]_flt_2_opt.SPC.fits : Optimally extracted version of the 1D spectra

While running the next notebook cell, it might take a few minutes to run

In [ ]:
os.chdir(cwd)
# infwhm (extrfwhm) & outfwhm (drzfwhm) can be (5,4)(wide), (4,3)(default) or (3,2)(narrow)
axetasks.axecore('aXe.lis',
                 config_file,
                 fconfterm=msky_file,
                 extrfwhm=4.,
                 drzfwhm=3.,
                 backfwhm=0.,
                 orient=False,
                 weights=True,
                 slitless_geom=False,
                 cont_model='gauss',
                 sampling='drizzle',
                 exclude=True)

Results are in the directory pointed to by os.environ['AXE_OUTPUT_PATH'], i.e. ./OUTPUT 1D and 2D spectra extracted from individual FLT files are available. These are not combined. SPC files contained 1D spectra, opt.SPC files contained optimally extracted spectra (using gaussian profiles), STP files contain 2D stamps. CONT files contain the contamination estimate (gaussian based)

## 10. Review Output Spectra

In [ ]:
os.chdir(cwd)
!ls OUTPUT/*SPC.fits
!ls OUTPUT/*STP.fits

In [ ]:
ID = 12 # object ID from the Source Extractor catalog

f1_FLT ="DATA/jdql01jjq_flt.fits"

f1_STP = "OUTPUT/jdql01jjq_flt_2.STP.fits"
f1_SPC = "OUTPUT/jdql01jjq_flt_2.SPC.fits"
f1_OPT_SPC = "OUTPUT/jdql01jjq_flt_2_opt.SPC.fits"

We can examine individual 2D spectra from the STP files. Note that the STP files are meant for quality control and are not calibrated versions of the 2D spectra.

In [ ]:
plt.rcParams["figure.figsize"] = (10,3)
try:
    d1 = fits.open(f1_STP)["BEAM_%dA" % (ID)].data
    im1 = plt.imshow(d1,origin="lower")
    im1.set_clim(0,400)
except:
    pass

We now examine the calibrated 1D spectra of one of the sources:

In [ ]:
for s in glob.glob(f1_SPC):
    print( s)
    d1 = fits.open(s)["BEAM_%dA" % (ID)].data
    w = d1["LAMBDA"]
    f = d1["FLUX"]
    e = d1["FERROR"]
    vg = (w>5500) & (w<10500)
    plt.errorbar(w[vg],f[vg],e[vg])
plt.xlabel(r'Wavelength ($\AA$)')
plt.ylabel(r'Flux ($erg/s/cm^2/\AA$)');

In [ ]:
for s in glob.glob(f1_SPC):
    print( s)
    d1 = fits.open(s)["BEAM_%dA" % (ID)].data
    w = d1["LAMBDA"]
    f = d1["COUNT"]
    vg = (w>5500) & (w<10500)
    plt.plot(w[vg],f[vg])
plt.xlabel(r'Wavelength ($\AA$)')
plt.ylabel(r'COUNT (Count)');

Contamination is not automatically removed but has been estimated and we can plot it

In [ ]:
for s in glob.glob(f1_SPC):
    print (s)
    d1 = fits.open(s)["BEAM_%dA" % (ID)].data
    w = d1["LAMBDA"]
    c = d1["CONTAM"]
    vg = (w>5500) & (w<12000)
    plt.plot(w[vg],c[vg],label=s)
plt.legend()
plt.xlabel(r'Wavelength ($\AA$)')
plt.ylabel(r'Flux ($erg/s/cm^2/\AA$)');

## 11. Additional Steps to Use 'aXedrizzle' Task to Combine Multiple Exposures

In [ ]:
os.chdir(cwd)
axetasks.drzprep(inlist = "aXe.lis", 
                 configs=config_file,
                 back = False,
                 opt_extr=True)

In [ ]:
!ls -altr OUTPUT/*DPP*

In [ ]:
os.chdir(cwd)
# infwhm (extrfwhm) & outfwhm (drzfwhm) can be (5,4)(wide), (4,3)(default) or (3,2)(narrow)
axetasks.axecrr(inlist="aXe.lis",
                configs=config_file,
                infwhm = 4.0,
                outfwhm = 3.0,
                back = False,
                driz_separate = 'yes',
                opt_extr=True)

The extraction results are in the DRIZZLE directory we created, and we can examine a 2D, rectified and wavelength calibrated version of the spectrum we looked at earlier:

In [ ]:
plt.rcParams["figure.figsize"] = (10,3)
d = fits.open("./DRIZZLE/aXedrizzle_2.STP.fits")["BEAM_%dA" % (ID)].data
im = plt.imshow(d)
im.set_clim(0,100)

We plot the extracted 1D spectra of our source and the estimate of the contamination:

In [ ]:
fin = fits.open("./DRIZZLE/aXedrizzle_2.SPC.fits")
tdata = fin["BEAM_%dA" % (ID)].data
x = tdata["LAMBDA"]
f = tdata["FLUX"]
e = tdata["FERROR"]

c = tdata["CONTAM"]
vg = (x>5500) & (x<10500)
plt.plot(x[vg],f[vg])
plt.errorbar(x[vg],f[vg],e[vg])

plt.plot(x[vg],c[vg])

The MEF files in the DRIZZLE directory contain the 2D version of the spectrum of a source as well as estimate of the contamination:

In [ ]:
plt.subplot(2,1,1)
d = fits.open("./DRIZZLE/aXedrizzle_mef_ID%d.fits" % (ID))["SCI"].data
im = plt.imshow(d)
im.set_clim(0,50)

plt.subplot(2,1,2)
d = fits.open("./DRIZZLE/aXedrizzle_mef_ID%d.fits" % (ID))["CON"].data
im = plt.imshow(d)
im.set_clim(0,10)

The individually extracted spectra are in the OUTPUT directory and the combined ones in the DRIZZLE directory. We can plot and compare them:

In [ ]:
for s in glob.glob(f1_SPC):
    print(s)
    d1 = fits.open(s)["BEAM_%dA" % (ID)].data
    w = d1["LAMBDA"]
    f = d1["FLUX"]
    e = d1["FERROR"]
    vg = (w>5500) & (w<10500)
    plt.errorbar(w[vg],f[vg],e[vg])
plt.xlabel(r'Wavelength ($\AA$)')
plt.ylabel(r'Flux ($erg/s/cm^2/\AA$)');


fin = fits.open("./DRIZZLE/aXedrizzle_2.SPC.fits")
tdata = fin["BEAM_%dA" % (ID)].data
x = tdata["LAMBDA"]
f = tdata["FLUX"]
e = tdata["FERROR"]

c = tdata["CONTAM"]
vg = (x>5500) & (x<10500)

plt.plot(x[vg],f[vg],color='k',lw=2)
plt.errorbar(x[vg],f[vg],e[vg],color='k',lw=2)

## Conclusions

Thank you for walking through this spectral extraction workflow. You should now be able to perform a basic extraction on ACS/WFC data using HSTaXe.

For detailed information on HSTaXe, please visit the [documentation webpage](https://hstaxe.readthedocs.io/en/latest/index.html).

Lastly, if you have questions regarding this notebook or using WFC3 data with HSTaXe please contact our [ACS Help Desk](https://stsci.service-now.com/hst).

## About this Notebook

**Author:** Nimish Hathi, ACS Instrument Branch (STScI)

**Updated On:** March 8, 2023